# Portfolio Optimization
## Data (Parameters)

- **i**: stock, $i = 1,2$.
- $\tilde r_i$: reward from stock $i$ (random variable).
- $\mu_i = \mathbb{E}[\tilde r_i]$: expected reward from stock $i$.
- $\mathrm{Var}(\tilde r_i)$: variance of reward from stock $i$.
- $\sigma_{ij} = \mathbb{E}\bigl[(\tilde r_i - \mu_i)(\tilde r_j - \mu_j)\bigr]$: covariance.  
  - Note that $\sigma_{ii} = \mathrm{Var}(\tilde r_i)$.
- **B**: budget for investment.
- **$\beta$**: target on expected portfolio reward.

## Decisions

- $x_i$: the amount to invest in stock $i$, $i = 1,2$.

## Objective

- Minimize total portfolio variance:
  
  $$
  \text{minimize}\quad \sum_{i=1}^2\sum_{j=1}^2 x_i\,\sigma_{ij}\,x_j
  $$

## Constraints

1. Expected reward is at least the target $\beta$:
   $$
   \sum_{i=1}^2 \mu_i\,x_i \;\ge\; \beta
   $$
2. Total investment does not exceed budget $B$:
   $$
   \sum_{i=1}^2 x_i \;\le\; B
   $$
3. No short sales:
   $$
   x_i \;\ge\; 0,\quad i=1,2
   $$


## Quadratic Optimization Model

$$
\begin{aligned}
&\min_{x} \;\mathrm{Var}\bigl(\tilde r_1 x_1 + \tilde r_2 x_2\bigr)\\
&\text{s.t.} \quad x_1 + x_2 \le B,\\
&\quad\quad\;\;\; \mu_1 x_1 + \mu_2 x_2 \ge \beta,\\
&\quad\quad\;\;\; x_i \ge 0,\; i=1,2.
\end{aligned}
$$

### Compact Form:
$$
\begin{aligned}
&\min \ \mathbf{x}^\top \mathbf{\Sigma} \mathbf{x} \\
\text{Subject to:} \\
&\mathbf{1}^\top \mathbf{x} \leq B \\
&\mathbf{\mu}^\top \mathbf{x} \geq \beta \\
&\mathbf{x} \geq \mathbf{0}
\end{aligned}
$$

## Is the objective linear?

$$
\begin{aligned}
\mathrm{Var}(\tilde r_1 x_1 + \tilde r_2 x_2)
&= \mathbb{E}\Bigl[\bigl((\tilde r_1-\mu_1)x_1+(\tilde r_2-\mu_2)x_2\bigr)^2\Bigr]\\
&= \sigma_{11}x_1^2 + 2\sigma_{12}x_1x_2 + \sigma_{22}x_2^2\\
&= \sum_{i=1}^2\sum_{j=1}^2 \sigma_{ij}\,x_i\,x_j\\
&= 
\begin{bmatrix}x_1 & x_2\end{bmatrix}
\begin{bmatrix}\sigma_{11} & \sigma_{12}\\ \sigma_{21} & \sigma_{22}\end{bmatrix}
\begin{bmatrix}x_1\\ x_2\end{bmatrix}.
\end{aligned}
$$

**Answer:** No – the objective is quadratic in \(x\).

## More Generally...

The variance of a linear combination of random variables is given by:

$$
\text{Var}(\hat{r}_1 x_1 + \cdots + \hat{r}_n x_n) = 
\begin{bmatrix} x_1 & \cdots & x_n \end{bmatrix}
\begin{bmatrix} \sigma_{11} & \cdots & \sigma_{1n} \\ \vdots & \ddots & \vdots \\ \sigma_{n1} & \cdots & \sigma_{nn} \end{bmatrix}
\begin{bmatrix} x_1 \\ \vdots \\ x_n \end{bmatrix}
= \mathbf{x}^\top \mathbf{\Sigma} \mathbf{x}
$$

- $\mathbf{\Sigma}$ is an $n \times n$ square matrix


## Covariance Matrix

- **$\mathbf{\Sigma}$: covariance matrix**
  - Symmetric matrix
  - Positive semidefinite matrix  
    $$
    \mathbf{x}^\top \mathbf{\Sigma} \mathbf{x} \geq 0, \quad \forall \mathbf{x} \in \mathbb{R}^n
    $$
  - The eigenvalues are real and nonnegative
  - There exists a matrix $\mathbf{A}$ such that $\mathbf{\Sigma} = \mathbf{A}^\top \mathbf{A}$

# Estimating $\mathbf{\Sigma}$ from Historical Data

- $I$ stocks, $T$ periods of data
- $\hat{r}_{ti}$: historical return of stock $i$ at period $t$
- $\mathbf{R} \in \mathbb{R}^{T \times I}$: matrix of $\hat{r}_{ti}$
- $\mu_i = \frac{1}{T} \sum_{t=1}^T \hat{r}_{ti}$: estimated mean reward
- Covariance matrix:

$$
\mathbf{\Sigma} = \frac{1}{T-1} (\mathbf{R} - 1 \mu^\top)^T (\mathbf{R} - 1 \mu^\top)
$$

where $\mu = \begin{bmatrix} \mu_1 \\ \vdots \\ \mu_I \end{bmatrix} \in \mathbb{R}^I$ and $1 = \begin{bmatrix} 1 \\ \vdots \\ 1 \end{bmatrix} \in \mathbb{R}^T$


In [3]:
import pandas as pd
from gurobipy import *

# Data loading
data1 = pd.read_csv("data_portfolio.csv")
stocks = data1.columns.values

# Calculate mean returns and covariance
ret_mean = data1.mean()
ret_mean = dict(ret_mean)
ret_cov = data1.cov()

# Set parameters
budget = 1000
exp_ret = 5

# Build model
m = Model("portfolio_mean_cov")

# Add variables: x[s] represents the amount invested in stock s
x = m.addVars(stocks, name="invest")

# Set objective: minimize portfolio variance
m.setObjective(
    quicksum(ret_cov.loc[i, j] * x[i] * x[j] for i in stocks for j in stocks),
    sense=GRB.MINIMIZE,
)

# Add constraints
m.addConstr(x.sum() <= budget, name="budget_con")  # Total investment within budget
m.addConstr(
    x.prod(ret_mean) >= exp_ret, name="exp_ret_con"
)  # Expected return constraint

# Write model to file and optimize
m.write("model.lp")
m.optimize()

# Print results
if m.status == GRB.OPTIMAL:
    for v in m.getVars():
        print(v.varName, v.x)
    print("portfolio variance =", m.objVal)
    stock_invested = [s for s in stocks if x[s].x > 1e-3]
    print(stock_invested)

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1360P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 5 columns and 10 nonzeros
Model fingerprint: 0x041be205
Model has 15 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-03, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e-04, 4e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 1e+03]
Presolve time: 0.01s
Presolved: 2 rows, 5 columns, 10 nonzeros
Presolved model has 15 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 4
 AA' NZ     : 1.500e+01
 Factor NZ  : 2.100e+01
 Factor Ops : 9.100e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   9.12157619e

## An alternative model

$$\begin{align*}
\max \quad & \mathbb{P}[\tilde{r}^{\top}x \geq \beta] \\
\text{s.t.} \quad & \mathbf{1}^{\top}x \leq B \\
& \boldsymbol{\mu}^{\top}x \geq \beta \\
& \mathbf{x} \geq \mathbf{0}
\end{align*}$$

- Different measures of risk
  - Variance v.s. probability of target attainment

## Solvability

- Generally intractable ☹
- Solvable under normal distribution assumption of $\tilde{r}$ ☺

$$\begin{align*}
\mathbb{P}[\tilde{r}^{\top}x \geq \beta] \\
= \mathbb{P}\left[\frac{\mu^{\top}x - \tilde{r}^{\top}x}{\sqrt{x^{\top}\Sigma x}} \leq \frac{\mu^{\top}x - \beta}{\sqrt{x^{\top}\Sigma x}}\right] \\
= \Phi\left(\frac{\mu^{\top}x - \beta}{\sqrt{x^{\top}\Sigma x}}\right)
\end{align*}$$

- $\Phi(\cdot)$: the cumulative distribution function of standard normal.

$$\begin{array}{ccc}
\max \quad \mathbb{P}(\tilde{r}'x \geq \beta) & \max \quad \Phi\left(\frac{\mu'x-\beta}{\sqrt{x'\Sigma x}}\right) & \max \quad \frac{\mu'x-\beta}{\sqrt{x'\Sigma x}} \\
\text{s.t.} \quad 1'x \leq B & \Leftrightarrow \text{s.t.} \quad 1'x \leq B & \Leftrightarrow \text{s.t.} \quad 1'x \leq B \\
\mu'x \geq \beta & \mu'x \geq \beta & \mu'x \geq \beta \\
x \geq 0 & x \geq 0 & x \geq 0
\end{array}$$

- Note: "$\Leftrightarrow$" in the sense that the models have the same optimal solutions
- Recall Sharpe ratio:
$$\frac{\mu^{\top}x - \beta}{\sqrt{x^{\top}\Sigma x}}$$

- Still intractable... Continue...

$$\begin{array}{ccc}
\max \quad \frac{\mu'x-\beta}{\sqrt{x'\Sigma x}} & \min \quad \frac{\sqrt{x'\Sigma x}}{\mu'x-\beta} & \min \quad \frac{x'\Sigma x}{(\mu'x-\beta)^2} \\
\text{s.t.} \quad 1'x \leq B & \Leftrightarrow \text{s.t.} \quad 1'x \leq B & \Leftrightarrow \text{s.t.} \quad 1'x \leq B \\
\mu'x \geq \beta & \mu'x \geq \beta & \mu'x \geq \beta \\
x \geq 0 & x \geq 0 & x \geq 0
\end{array}$$

- Suppose there exists an optimal solution such that $\mu^T x > \beta$, i.e., $y > 0$.

$$\begin{array}{ccc}
\min \quad \frac{x'\Sigma x}{(\mu'x - \beta)^2} & \min \quad \frac{x'\Sigma x}{y^2} & \min \quad \frac{x'}{y}\Sigma\frac{x}{y} \\
\text{s.t.} \quad 1'x \leq B & \Leftrightarrow \text{s.t.} \quad 1'x \leq B & \Leftrightarrow \text{s.t.} \quad 1'\frac{x}{y} \leq B\frac{1}{y} \\
\mu'x \geq \beta & \mu'x - \beta = y & \mu'\frac{x}{y} - \beta\frac{1}{y} = 1 \\
x \geq 0 & y \geq 0 & \frac{1}{y} \geq 0 \\
& x \geq 0 & \frac{x}{y} \geq 0
\end{array}$$

- Change of variables (trick!)
 - Let $\bar{x} = x/y$, $z = 1/y$

$$\begin{array}{cc}
\min \quad \frac{x'}{y}\Sigma\frac{x}{y} & \min \quad \bar{x}'\Sigma\bar{x} \\
\text{s.t.} \quad 1'\frac{x}{y} \leq B\frac{1}{y} & \Leftrightarrow \text{s.t.} \quad 1'\bar{x} \leq Bz \\
\mu'\frac{x}{y} - \beta\frac{1}{y} = 1 & \mu'\bar{x} - \beta z = 1 \\
\frac{1}{y} \geq 0 & z \geq 0 \\
\frac{x}{y} \geq 0 & \bar{x} \geq 0
\end{array}$$

Solve the quadratic optimization problem:

$$\begin{align*}
\min \quad & \bar{x}'\Sigma\bar{x} \\
\text{s.t.} \quad & 1'\bar{x} \leq Bz \\
& \mu'\bar{x} - \beta z = 1 \\
& z \geq 0 \\
& \bar{x} \geq 0
\end{align*}$$

Actual portfolio weights: $x = \bar{x}y = \bar{x}/z$

In [ ]:
import pandas as pd
import numpy as np
from gurobipy import *

# Data loading and preprocessing
data = pd.read_csv("data_portfolio.csv")
stocks = data.columns.values

# Calculate mean returns and covariance matrix
mu = data.mean()
Sigma = data.cov()

# Set parameters
B = 1000  # budget
beta = 5  # target return

# Create model
m = Model("portfolio_prob_max")

# Add variables
x_bar = m.addVars(stocks, name="x_bar")
z = m.addVar(name="z")

# Set objective: minimize x'Σx
obj = quicksum(Sigma.loc[i, j] * x_bar[i] * x_bar[j] for i in stocks for j in stocks)
m.setObjective(obj, GRB.MINIMIZE)

# Add constraints
# Budget constraint: 1'x_bar ≤ Bz
m.addConstr(quicksum(x_bar[s] for s in stocks) <= B * z, name="budget")

# Expected return constraint: μ'x_bar - βz = 1
m.addConstr(quicksum(mu[s] * x_bar[s] for s in stocks) - beta * z == 1, name="return")

# Non-negativity constraints
m.addConstr(z >= 0, name="z_nonneg")
for s in stocks:
    m.addConstr(x_bar[s] >= 0, name=f"x_{s}_nonneg")

# Solve model
m.optimize()

# Output results
if m.status == GRB.OPTIMAL:
    print("\nOptimal Solution:")
    # Calculate actual portfolio weights x = x_bar/z
    z_val = z.x
    portfolio = {}
    for s in stocks:
        x_val = x_bar[s].x / z_val
        if x_val > 1e-6:  # Only print significant investments
            portfolio[s] = x_val
            print(f"{s}: {x_val:.4f}")

    # Calculate portfolio statistics
    weights = np.array([portfolio.get(s, 0) for s in stocks])
    exp_return = np.dot(mu, weights)
    risk = np.sqrt(np.dot(weights.T, np.dot(Sigma, weights)))

    print(f"\nPortfolio Statistics:")
    print(f"Total Investment: {sum(portfolio.values()):.2f}")
    print(f"Expected Return: {exp_return:.4f}")
    print(f"Risk (Std Dev): {risk:.4f}")
    print(f"Sharpe Ratio: {(exp_return - beta)/risk:.4f}")
else:
    print("Model optimization failed")

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1360P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 8 rows, 6 columns and 18 nonzeros
Model fingerprint: 0xc716fab8
Model has 15 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-03, 1e+03]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e-04, 4e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 7 rows and 1 columns
Presolve time: 0.01s
Presolved: 1 rows, 5 columns, 5 nonzeros
Presolved model has 15 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 4
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective          